In [1]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule_multigpu

import pyNeuroChem as pync
import pyaniasetools as pya
import hdnntools as hdt

import numpy as np
import  ase

import time

import os
os.environ["OMP_NUM_THREADS"] = "8"

import  ase
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units

from ase.optimize.fire import FIRE as QuasiNewton
from ase.optimize import LBFGS

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

from ase.io import read, write

from ase.parallel import world

van Der Waals correction will be unavailable. Please install ased3


In [2]:
dir = '/nh/nest/u/jsmith/scratch/Research/MD_TEST/fspeptide/'

# Molecule file
molfile = dir + 'fspeptide_solv.pdb'

# Dynamics file
xyzfile = dir + 'mdcrd.xyz'

# Trajectory file
trajfile = dir + 'traj.dat'

# Optimized structure out:
optfile = dir + 'ani_optmol.xyz'

#T = 300.0 # Temperature
dt = 0.2
C = 0.1 # Optimization convergence
#steps = 1000

ntdir = '/nh/nest/u/jsmith/scratch/Research/MD_TEST/fspeptide/model-ani2x-12/'
cns = ntdir + 'train0/rHCNOSFCl-5.0R_16-3.4A_a4-8.params'
sae = ntdir + 'train0/sae_linfit.dat'
nnf = ntdir + 'train'
Nn = 8

In [5]:
# Load molecule
mol = read(molfile)
# L = 40.0
mol.set_cell(([[100.0, 0, 0],
               [0, 35.0, 0],
               [0, 0, 35.0]]))

mol.set_pbc((True, True, True))

# Set NC
aens = ensemblemolecule_multigpu(cns, sae, nnf, Nn, [0, 1])

# Set ANI calculator
mol.set_calculator(ANIENS(aens))

s_time = time.time()
E = mol.get_potential_energy()
F = mol.get_forces()
print('Total time:',time.time()-s_time)

spc = mol.get_chemical_symbols()
print(len(spc))

Building...
Building...
Total time: 4.126553535461426
12530


Process Process-4:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/scratch/Anaconda/anaconda3/envs/my-rdkit-env/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/scratch/Anaconda/anaconda3/envs/my-rdkit-env/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/scratch/Gits/NeuroChem/build/lib/ase_interface.py", line 164, in molecule_worker
    next_task = task_queue.get()
  File "/scratch/Anaconda/anaconda3/envs/my-rdkit-env/lib/python3.6/multiprocessing/queues.py", line 94, in get
    res = self._recv_bytes()
  File "/scratch/Anaconda/anaconda3/envs/my-rdkit-env/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/scratch/Anaconda/anaconda3/envs/my-rdkit-env/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/scratch/Anaconda/anaconda3/envs/

In [6]:
# Optimize molecule
start_time = time.time()
dyn = LBFGS(mol)
dyn.run(fmax=C)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

# Save optimized mol
spc = mol.get_chemical_symbols()
pos = mol.get_positions(wrap=True).reshape(1,len(spc),3)


hdt.writexyzfile(optfile, pos, spc)

       Step     Time          Energy         fmax
LBFGS:    0 15:33:33 -8672018.336271        7.4624
LBFGS:    1 15:33:34 -8672247.475244        4.5574
LBFGS:    2 15:33:35 -8672428.215021        4.8358
LBFGS:    3 15:33:37 -8672519.710649        5.1679
LBFGS:    4 15:33:38 -8672589.298751        6.8674
LBFGS:    5 15:33:39 -8672641.879867        7.6863
LBFGS:    6 15:33:40 -8672696.003359        6.7459
LBFGS:    7 15:33:41 -8672739.070394       15.8064
LBFGS:    8 15:33:42 -8672778.470722       18.1066
LBFGS:    9 15:33:44 -8672816.289293       29.4770
LBFGS:   10 15:33:46 -8672853.239412       22.7173
LBFGS:   11 15:33:47 -8672882.333327       21.7224
LBFGS:   12 15:33:48 -8672909.131413       26.4237
LBFGS:   13 15:33:49 -8672946.095005       26.2526
LBFGS:   14 15:33:51 -8672984.009702       20.2370
LBFGS:   15 15:33:52 -8673021.279791       14.7973
LBFGS:   16 15:33:53 -8673066.111154       13.4896
LBFGS:   17 15:33:54 -8673107.251550       14.3813
LBFGS:   18 15:33:55 -8673142.16

KeyboardInterrupt: 

In [ ]:
# We want to run MD with constant energy using the VelocityVerlet algorithm.
#dyn = NVTBerendsen(mol, 1 * units.fs, 100.0, taut=1.0*1000*units.fs)
# coefficient to 0.04 atomic units.
dyn = Langevin(mol, dt * units.fs, 1.0 * units.kB, 0.04)

In [ ]:
# Open MD output
mdcrd = open(xyzfile,'w')

# Open MD output
traj = open(trajfile,'w')

# Define the printer
def storeenergy(a=mol, d=dyn, b=mdcrd, t=traj):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)

    stddev =  hdt.evtokcal*a.calc.stddev

    t.write(str(d.get_number_of_steps()) + ' ' + str(ekin / (1.5 * units.kB)) + ' ' + str(epot) + ' ' + str(ekin) + ' ' + str(epot+ekin) + ' ' + str(stddev) + '\n')
    b.write(str(len(a)) + '\n' + str(ekin / (1.5 * units.kB)) + ' Step: ' + str(d.get_number_of_steps()) + '\n')
    c = a.get_positions(wrap=True)
    for j, i in zip(a, c):
        b.write(str(j.symbol) + ' ' + str(i[0]) + ' ' + str(i[1]) + ' ' + str(i[2]) + '\n')

    print('Step: %d Energy per atom: Epot = %.3f  Ekin = %.3f (T=%.3fK)  '
          'Etot = %.6f Esig = %.6f' % (d.get_number_of_steps(), epot, ekin, ekin / (1.5 * units.kB), epot + ekin, stddev))
    
# Set the printer
dyn.attach(storeenergy, interval=1000)

In [ ]:
dyn.set_temperature(50. * units.kB)
dyn.run(25000)

dyn.set_temperature(100. * units.kB)
dyn.run(25000)

dyn.set_temperature(150. * units.kB)
dyn.run(25000)

dyn.set_temperature(200. * units.kB)
dyn.run(25000)

dyn.set_temperature(250. * units.kB)
dyn.run(25000)

dyn.set_temperature(300. * units.kB)
dyn.run(25000)

dyn.set_temperature(325. * units.kB)
dyn.run(25000)

start_time = time.time()
dyn.set_temperature(325. * units.kB)
dyn.run(250000000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')